In [1]:
from etherscan.accounts import Account
from api_key import key
import pandas as pd
from datetime import datetime, timedelta
import json

ImportError: No module named 'api_key'

In [21]:
api = Account()

In [3]:
transactions = api.get_all_transactions()

page 1 added
page 2 added
page 3 added
page 4 added
page 5 added
page 6 added
page 7 added
page 8 added
page 9 added
page 10 added
page 11 added
Total number of transactions: 103457


In [36]:
# Write to file to keep it up to date
with open('dao_transactions.json', mode='w') as f:
    f.write(transactions)

In [2]:
trans = pd.read_json('dao_transactions.json')

In [41]:
# trans = pd.DataFrame(transactions)
trans['value'] = trans.value.astype(float)
trans['blockNumber'] = trans.blockNumber.astype(int)
trans['value_ether'] = trans.value / 1e18
trans['gasPrice'] = trans.gasPrice.astype(float)
trans['gasUsed'] = trans.gasUsed.astype(float)
trans['value_cum'] = trans.value_ether.cumsum()
trans.cumulativeGasUsed = trans.cumulativeGasUsed.astype(float)
trans = trans.set_index('timeStamp')

In [39]:
dao_end_time = datetime(2016, 5, 28, 9, 0, 0, 0)
dao_phase_one_end = dao_end_time - timedelta(days=14)
dao_phase_two_end = dao_end_time - timedelta(days=5)

In [49]:
creation = trans[trans.index <= dao_end_time]
movement = trans[trans.index > dao_end_time]

Curator Information: 

In [ ]:
exchanges = dict(
    poloniex = '0xb794f5ea0ba39494ce839613fffba74279579268',
    polo_2 = '0x32be343b94f860124dc4fee278fdcbd38c102d88',
    polo_new = '0xDf21fA922215B1a56f5a6D6294E6E36c85A0Acfb',
    kraken = '0x2910543af39aba0cd09dbb2d50200b3e800a63d2',
    bitfinex = '0xcafb10ee663f465f9d10588ac44ed20ed608c11e',  
    gatecoin_hacked = '0x40b9b889a21ff1534d018d71dc406122ebcf3f5a',
)

In [4]:
curator_contract_address = '0xda4a4626d3e16e094de3225a751aab7128e96526'

curators = {
'Vitalik Buterin':'1db3439a222c519ab44bb1144fc28167b4fa6ee6',
'Aeron Buchanan':'b2c1b92f4bed7a173547cc601fb73a1254d10d26',
'Christian Reitwießner':'0029218e1dab069656bfb8a75947825e7989b987',
'Taylor Gerring':'cee96fd34ec793b05ee5b232b0110eac0cc3327e',
'Viktor Tron':'b274363d5971b60b6aca27d6f030355e9aa2cf23',
'Fabian Vogelsteller':'c947faed052820f1ad6f4dda435e684a2cd06bb4',
'Martin Becze':'ae90d602778ed98478888fa2756339dd013e34c1',
'Gustav Simonsson':'e578fb92640393b95b53197914bd560b7bc2aac8',
'Vlad Zamfir':'127ac03acfad15f7a49dd037e52d5507260e1425',
'Gavin Wood':'0037a6b811ffeb6e072da21179d11b1406371c63',
'Alex Van de Sande':'d1220a0cf47c7b9be7a2e6ba89f429762e7b9adb',
'Iuri Matias':'C157f767030B4cDd1f4100e5Eb2b469b688D293E',
'Shermin Voshmgir':'820c6da74978799d574f61b01f8b5eebc051f95e',
'Gavin Wood':'0037a6b811ffeb6e072da21179d11b1406371c63',
}

In [2]:
function_codes_str = [
'643f7cdd=DAOpaidOut',
'82bf6464=DAOrewardAccount',
'39d1f908=actualBalance',
'dd62ed3e=allowance',
'4df6d6cc=allowedRecipients',
'095ea7b3=approve',
'70a08231=balanceOf',
'e5962195=blocked',
'749f9889=changeAllowedRecipients',
'e33734fd=changeProposalDeposit',
'eceb2945=checkProposalCode',
'4b6753bc=closingTime',
'baac5300=createTokenProxy',
'e66f53b7=curator',
'149acf9a=daoCreator',
'1f2dc5ef=divisor',
'237e9492=executeProposal',
'21b5b8dd=extraBalance',
'cc9ae3f6=getMyReward',
'be7c29c1=getNewDAOAddress',
'78524b2e=halveMinQuorum',
'b7bc2c84=isFueled',
'96d7f3f5=lastTimeMinQuorumMet',
'674ed066=minQuorumDivisor',
'0c3b7b96=minTokensToCreate',
'6837ff1e=newContract',
'612e45a3=newProposal',
'8d7af473=numberOfProposals',
'81f03fcb=paidOut',
'f8c80d26=privateCreation',
'8b15a605=proposalDeposit',
'013cf08b=proposals',
'a3912ec8=receiveEther',
'590e1ae3=refund',
'a1da2fb9=retrieveDAOReward',
'0e708203=rewardAccount',
'cdef91d0=rewardToken',
'82661dc4=splitDAO',
'34145808=totalRewardToken',
'18160ddd=totalSupply',
'a9059cbb=transfer',
'23b872dd=transferFrom',
'dbde1988=transferFromWithoutReward',
'4e10c3ee=transferWithoutReward',
'2632bf20=unblockMe',
'c9d27afe=vote',
]

dao_fn_codes = dict()
for string in function_codes_str:
    code, function = string.split('=')
    dao_fn_codes[function] = str(code)

In [32]:
def check_for_function(string: str, function: str):
    called = []
    for key, value in dao_fn_codes.items():
        if value in string:
            called.append(key)
    return ''.join(called)
    
    
trans['functions'] = [check_for_function(string, dao_fn_codes) for string in trans['input'].values]
trans.head()

,blockHash,blockNumber,confirmations,contractAddress,cumulativeGasUsed,from,gas,gasPrice,gasUsed,hash,...,isError,isInternalTx,nonce,timeStamp,to,transactionIndex,value,value_ether,value_cum,functions
0,0x9a9d1fc252e816fd047b85b6c1028317ecc0bf3db18d...,1429008,252939,,47080.0,0x931f12be72a1d106408af5ced7e1b7af0bc864ae,49900,2.000000e+10,47080.0,0x18aa650060a1434310a05824e1df6e29a6904f29839e...,...,0,0,0,2016-04-30 02:44:37,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.000000e+00,0.00,0.00,approve
1,0x9a9d1fc252e816fd047b85b6c1028317ecc0bf3db18d...,1429008,252939,,136160.0,0xb504e60998c6f354a0794abd91d85e8bc8436211,49900,2.000000e+10,47080.0,0xe71a87b83e054f616e8b1f4884b905d9a347b64433bb...,...,0,0,0,2016-04-30 02:44:37,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,3,0.000000e+00,0.00,0.00,approve
10,0x031d5bac6154ca7616ac62e966da2b50a0aaa1b3bc24...,1429038,252909,,284538.0,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,241206,2.000000e+10,91206.0,0xc96b0f95a1e7e8c07cd488a05f20f9e8d4003fe8eea0...,...,0,0,11347,2016-04-30 02:52:27,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,9,3.000000e+16,0.03,0.03,createTokenProxy
100,0x6406520c939bccabc937b84347b4a959563a75031ee8...,1429120,252827,,162240.0,0xe0f801a09a420c3e2ae47caaee152646dce304ee,49900,2.000000e+10,47080.0,0x20a77d2a39a2c68d4ffd9dd15690f474e83bf7745751...,...,0,0,0,2016-04-30 03:07:57,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,3,0.000000e+00,0.00,0.03,approve
1000,0x174d4769f1c995733501e392d39ec7c6408e6c3c003b...,1432199,249748,,149091.0,0x2a2107f530ff99709d2e19c7bb8622027163cf9e,100000,2.100000e+10,44091.0,0xd729b4aedfe65030e2f972bc83fdb1215b515c7710c7...,...,0,0,2,2016-04-30 15:12:35,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,5,6.000000e+18,6.00,6.03,


In [34]:
trans[trans['functions'].isin(['splitDAO'])]

,blockHash,blockNumber,confirmations,contractAddress,cumulativeGasUsed,from,gas,gasPrice,gasUsed,hash,...,isError,isInternalTx,nonce,timeStamp,to,transactionIndex,value,value_ether,value_cum,functions
100180,0xba02a064ab9406467d45d5ac605f24d787790df99221...,1661021,20927,,3048507.0,0xe7535ddfcbefe5c318d271476d068d5f7cf77290,3148507,2.000000e+10,3048507.0,0x3e887165b6f9958ce8c2b46f46fb4c8c45b1605aedd3...,...,0,0,9,2016-06-07 15:27:21,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,4.445001e+03,splitDAO
100617,0x0e1ae8cd2f16d5c198ee51530c6748b0595530c494b9...,1663755,18193,,3048507.0,0x6c0d74c64b4ed871837651c4ab3cdce425c1ec6c,3148507,2.000000e+10,3048507.0,0x724a4cafe12b7c446cb5cba7faed334804ad92043065...,...,0,0,5,2016-06-08 02:16:36,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,2.836648e+04,splitDAO
100648,0x81f01450f2be7a8dd456fdae4ce4eb4d801e5d989700...,1663977,17971,,3411779.0,0x95a61f934d66580dd410a7369f9c5b8e228d2ff3,3133619,2.000000e+10,3033619.0,0x361c0a9e9c211acceb9472afd130f49bb1ecbfdfe540...,...,0,0,3,2016-06-08 03:11:23,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,18,0.0,0.0,2.838758e+04,splitDAO
100650,0x2d971c29d579eb1d1dc00db9be32a4484edd15a55b9d...,1664014,17934,,3033507.0,0xb18e6467db64686dfed14c7368ca59e5019c95c8,3133507,2.000000e+10,3033507.0,0x5ae59075e0e400e5f2a8c610ca2c51ee7e25bcc1ccef...,...,0,0,3,2016-06-08 03:17:49,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,2.843511e+04,splitDAO
100866,0x4e96d75a20f28bf5e9f04bfcbd3202c1f9b6abe21a06...,1665558,16390,,3112767.0,0x598c72e3fe70e76d2e2f47f529f22634330ffcf3,3178507,2.000000e+10,3048507.0,0x875eba49502ece7ccf6e319662721f538150ca843069...,...,0,0,1,2016-06-08 09:30:22,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,2,0.0,0.0,3.004059e+04,splitDAO
100996,0xb69006a8779bbfa81f3140d0011f02485e79a9acedba...,1666381,15567,,3033507.0,0xb42da5b3701a0592e5aa0aebc0c20711bd49fb46,3133507,3.206417e+10,3033507.0,0xc0745839be871a5a84f1c16de8609aaeccca2a8467fd...,...,0,0,3,2016-06-08 12:53:10,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,3.081699e+04,splitDAO
101119,0x3e9075ba681cdc2972214156478b381a4c44df4db4ed...,1667056,14892,,3033507.0,0xcf69ab35bb6a87a68ce83571a174eef4f998baa7,3200000,1.000000e+11,3033507.0,0xe0ab9a936821a45fb857cb19a8164c6a3a1ba29ddbf5...,...,0,0,3,2016-06-08 15:29:23,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,3.139072e+04,splitDAO
101175,0xd8bd1db17a37b7c861d4439939cf1901afe811c7a5e1...,1667381,14567,,82538.0,0xcf69ab35bb6a87a68ce83571a174eef4f998baa7,3200000,1.000000e+11,82538.0,0x147099854eb6929becab3a4f2580f8c4c16bcc93027f...,...,0,0,4,2016-06-08 16:49:03,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,3.151107e+04,splitDAO
101221,0x06582b51434f4194707625b294b88c64335a28571965...,1667694,14254,,723752.0,0xcf69ab35bb6a87a68ce83571a174eef4f998baa7,320000,2.000000e+10,82538.0,0x4de12e26ba3471aaaf38b8be314450a087285c5e37b0...,...,0,0,5,2016-06-08 17:56:58,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,21,0.0,0.0,3.255631e+04,splitDAO
101440,0x6853eec643b8c708329904da54047c5be23513bad386...,1668800,13148,,3118767.0,0xfdf97eaa34a883647fac329926b1747e9ef601c6,3163507,2.000000e+10,3033507.0,0x74a2d007d74879ea811c4606c08ec73930336b58fcc4...,...,0,0,12,2016-06-08 22:15:10,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,3,0.0,0.0,3.456105e+04,splitDAO


In [50]:
def token_rate(timeStamp):
    token_rate = [1.0 + 0.05*(x+1) for x in range(10)]
    days = [datetime(2016, 5, 28, 9, 0, 0, 0) - timedelta(days=13) + timedelta(days=1)*x for x in range(10)]
    
    for rate, day in zip(token_rate,days):
        if timeStamp < days[0] - timedelta(days=1):
            return 1.0
        elif day < timeStamp < day + timedelta(days=1):
            return rate
    return 1.5

creation['token_rate'] = creation.reset_index().timeStamp.apply(token_rate)
creation['tokens'] = creation.value_ether / creation.token_rate * 100

/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [51]:
cum_ether = pd.DataFrame(creation.groupby('from').value_ether.sum().reset_index(), columns=['from', 'value_ether'])
cum_tokens = pd.DataFrame(creation.groupby('from').tokens.sum().reset_index(), columns=['from','tokens'])
cum_trans = creation.groupby('from').size().sort_values().reset_index()
cum_trans.columns = ['from', 'num_trans']
grouped = pd.merge(cum_ether, cum_trans, on='from')
grouped = pd.merge(grouped, cum_tokens, on='from')

In [52]:
# pd.merge(trans[trans['isSplit']][['timeStamp', 'from']], grouped, on='from')

In [54]:
creation[creation['functions'].isin(['transfer'])]

,blockHash,blockNumber,confirmations,contractAddress,cumulativeGasUsed,from,gas,gasPrice,gasUsed,hash,...,isInternalTx,nonce,to,transactionIndex,value,value_ether,value_cum,functions,token_rate,tokens
timeStamp,,,,,,,,,,,,,,,,,,,,,
2016-05-05 00:59:55,0x157ada30eb9bdc6843d984f5698b7cb64aab41314cb2...,1458665,223282,,2139820.0,0x9296f782bd381b67e3ff3a8cf294069f293eb2f3,121000,2.000000e+10,121000.0,0x7c1b67b06133aa2c97eff0746ef28779a6ab9f23f81b...,...,0,7,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,1,0.0,0.0,2.656495e+04,transfer,NaN,NaN
2016-05-05 01:59:25,0x16f70d6ceaa5c24ad8dc97e63d2b18a6d04ffc77aafc...,1458920,223027,,163000.0,0x9296f782bd381b67e3ff3a8cf294069f293eb2f3,121000,2.000000e+10,121000.0,0x77f438ca731899a8dee88a766811f6822208d9201426...,...,0,8,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,2,0.0,0.0,2.996979e+04,transfer,NaN,NaN
2016-05-05 02:09:40,0xc1273df36a41fb8fcaafdff481c003938da5bf1c2682...,1458957,222990,,142000.0,0xa282e969cac9f7a0e1c0cd90f5d0c438ac570da3,121000,2.000000e+10,121000.0,0x797557d999de98637caf144ce5c9b347f10049ecb265...,...,0,8,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,1,0.0,0.0,3.004059e+04,transfer,NaN,NaN
2016-05-05 04:53:59,0x58ce33a43a20ce8212e2a447537c14f584ede0089be4...,1459644,222303,,121000.0,0x174270bcbc535e9134cb496065c3a2121511cf41,121000,2.200000e+10,121000.0,0x7cd162af4e9ec0dd7c7737f53bd3859fa3484d4bcfa7...,...,0,1,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,4.816765e+04,transfer,NaN,NaN
2016-05-05 05:40:07,0x17424921294d5c07c637aaa326a7b514d15bd23b46f3...,1459852,222095,,142000.0,0x2898f77df306e8caa1d6c2e7be418772f93fe8de,121000,2.000000e+10,121000.0,0x781d5299d57e7b6b6bae1a927c8bcbfe35b7527f12e0...,...,0,11,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,1,0.0,0.0,5.570721e+04,transfer,NaN,NaN
2016-05-05 22:06:56,0x94752fdf2a011bbb1d125816160048089437c342d646...,1464026,217921,,373160.0,0xf883c87561bbb16a281e2d247ef61fee1f6a4f2a,121000,2.000000e+10,121000.0,0x0b9e8910cbc94d8e48f53df212a0371ef668faeeeec5...,...,0,7,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,12,0.0,0.0,3.185970e+05,transfer,NaN,NaN
2016-05-06 00:57:15,0x10828e9154da97b9ece13235b3a46db8e66a21981217...,1464706,217241,,331000.0,0x64ba5743b5a78ec1df4b4c00013050974247e82d,121000,2.000000e+10,121000.0,0x7e0a54b6ba906344b03af1b7223acf5b360bd2637fa0...,...,0,1,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,10,0.0,0.0,3.380740e+05,transfer,NaN,NaN
2016-05-06 03:21:26,0x232231cbb540e9cd76fe7f81d9375b25b4296a33ecbd...,1465297,216650,,289000.0,0x3dd50d18fda8f9955a5c99706790b87410e59efb,121000,1.818200e+10,121000.0,0x18c49224090392f4743940e1c00dcc57ce918889bee8...,...,0,2,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,8,0.0,0.0,3.557216e+05,transfer,NaN,NaN
2016-05-06 07:33:16,0x7a12607d85031bbc90b91b8953d43023aa32f83b2ab9...,1466345,215602,,289000.0,0x66135f35aff788ca7c0ee235732a3180d2e841ef,121000,1.818182e+10,121000.0,0x514b47f0354f0cb5a25c6ba595bd3705d6a22cf6efd8...,...,0,2,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,8,0.0,0.0,3.994916e+05,transfer,NaN,NaN
